# Import Packages

In [ ]:
# import labelme2yolo 
import os
import gc
import shutil
from tqdm import tqdm
from sklearn.model_selection import *
from ultralytics import YOLO
import warnings
import cv2
warnings.filterwarnings('ignore')

# Data Preprocess 

In [ ]:
"""Converting all the JSON labels to YOLO format"""
!labelme2yolo --json_dir "C:\\Users\\Eurus\\Desktop\\YoloV8\\L"

In [ ]:
# # Collecting all the correspounding addresses for the Pics
image_size = 540

def image_divider(image_dir):
    images_path = []
    # Reads all the paths to the images
    for idx in tqdm(os.listdir(image_dir)):
        if idx.endswith('.jpg'):
            if "T" in idx:
                images_path.append({"Address":idx, "Label":"Y"})
            elif "N" in idx:
                images_path.append({"Address":idx, "Label":"N"})
            else:
                print("Error! Something went wrong...")
                print(idx)

    return images_path


In [ ]:
img_data_path = image_divider("C:\\Users\\Eurus\\Desktop\\YoloV8\\images")

In [ ]:
# Double Checking
img_data_path[12610]

In [ ]:
# Splitting the data into Train/Validation/Test Sets with respect to their distribution
train_images_indexes, test_images_init_indexes = train_test_split(img_data_path, test_size=0.3, stratify=[d['Label'] for d in img_data_path])

In [ ]:
# This time divide the test set in two sets of TEST and VALIDATION with respect to the general rule of label distribution
validation_images_indexes, test_images_indexes = train_test_split(test_images_init_indexes, test_size=0.5, stratify=[d['Label'] for d in test_images_init_indexes])

In [ ]:
# No more needed since we have our three sets
del test_images_init_indexes

In [ ]:
"double checking the distribution of labels in Validation/Training/Test set"
Tumors = 0
Normals = 0

for idx in range(len(train_images_indexes)):
    if train_images_indexes[idx]['Label']=="Y":
        Tumors+=1
    elif train_images_indexes[idx]['Label']=="N":
        Normals+=1
        
Tumors = Tumors*100/len(train_images_indexes) 

print(f"Total percentage of Tumor cases : {Tumors}\n")

* Total percentage of Tumor cases in Training Set:    
    * %9.19

* Total percentage of Tumor cases in Validation Set:
    * %9.20

* Total percentage of Tumor cases in Test Set:        
    * %9.15

In [ ]:
# Simply remove the label Key from the Dictionary
def cleaner(my_input):
    for idx in range(len(my_input)):
        del my_input[idx]['Label']


In [ ]:
cleaner(train_images_indexes)
cleaner(validation_images_indexes)
cleaner(test_images_indexes)

In [ ]:
train_images_indexes[0]

In [ ]:
# Move all of them to their correspouding folder
training_path = "C:\\Users\\Eurus\\Desktop\\YoloV8\\train\\images"
validation_path = "C:\\Users\\Eurus\\Desktop\\YoloV8\\val\\images"
test_path = "C:\\Users\\Eurus\\Desktop\\YoloV8\\test\\images"

def move_images(root, destination):
    main_dir = "C:\\Users\\Eurus\\Desktop\\YoloV8\\images"
    for index in tqdm(range(len(root))):
        temp = os.path.join(main_dir, root[index]['Address'])
        shutil.move(temp, destination)
        del temp

In [ ]:
move_images(train_images_indexes, training_path)

In [ ]:
move_images(validation_images_indexes, validation_path)

In [ ]:
move_images(test_images_indexes, test_path)

In [ ]:
# Clear the JPG from their names to further locate them in labels folder
def cleaner_2(my_list):
    for idx in range(len(my_list)):
        my_list[idx]['Address'] = my_list[idx]['Address'].rstrip(".jpg")
        

In [ ]:
cleaner_2(test_images_indexes)

In [ ]:
cleaner_2(validation_images_indexes)

In [ ]:
cleaner_2(train_images_indexes)

In [ ]:
# now move the labels ...
training_label_path = "C:\\Users\\Eurus\\Desktop\\YoloV8\\train\\labels"
validation_label_path = "C:\\Users\\Eurus\\Desktop\\YoloV8\\val\\labels"
test_label_path = "C:\\Users\\Eurus\\Desktop\\YoloV8\\test\\labels"

def move_labels(root, destination):
    main_dir = "C:\\Users\\Eurus\\Desktop\\YoloV8\\labels"
    for index in tqdm(range(len(root))):
        # Only the tumors have labels
        if "T" in root[index]['Address']:
            temp = os.path.join(main_dir, root[index]['Address']+".txt")
            shutil.move(temp, destination)
            del temp

In [ ]:
move_labels(train_images_indexes,training_label_path)

In [ ]:
move_labels(validation_images_indexes,validation_label_path)

In [ ]:
move_labels(test_images_indexes,test_label_path)

In [ ]:
del test_images_indexes, validation_images_indexes, train_images_indexes

In [ ]:
gc.collect()

Need to write a function to create dummy coordinates for images without Tumor

In [ ]:
def coordinator(path):
    for idx in tqdm(os.listdir(path)):
        if "N" in idx:
            idx = idx.rstrip(".jpg")
            temp_file = open(f"{idx}.txt", "x")
            # add dummy but consistent coordinates
            temp_file.write("1 0.500926 0.500926 0.998148 0.998148")
            temp_file.close()
        else:
            pass

In [ ]:
path = "C:\\Users\\Eurus\\Desktop\\YoloV8\\train\\images"

coordinator(path)

In [ ]:
path = "C:\\Users\\Eurus\\Desktop\\YoloV8\\val\\images"

coordinator(path)

# Training

In [ ]:
model = YOLO("yolov8n.pt")

In [ ]:
model.train(data="C:\\Users\\Eurus\\Desktop\\YoloV8\\custom_data.yaml",
            epochs=50,
            batch=-1,
            imgsz=540,
            verbose=True,
            amp=False,
            device=0,
#             classes=['Tumor'],
#             single_cls=True
           )

# Test

In [ ]:
model.predict("C:\\Users\\Eurus\\Desktop\\Test_Data\\Normal\\Patient-970504191301",
             save=True,
             device=0,
             save_conf=True,
             save_txt=True,
             max_det=5
            )

In [ ]:
# Making predictions Case By Case
root = "C:\\Users\\Eurus\\Desktop\\Test_Data\\Normal"
destination = "D:\\NormalCases" 

def perform_detection(root, destination):
    const = "C:\\Users\\Eurus\\Desktop\\YoloV8\\runs\\detect\\predict"
    for index in os.listdir(root):
        patient_root = os.path.join(root,index)
        patient_dest = os.path.join(destination,index)
        os.makedirs(patient_dest, exist_ok=True)
        
        for image in tqdm(os.listdir(patient_root)):
            temp_path = os.path.join(patient_root,image)
            # Make detections
            detections = model.predict(source=temp_path,
                                       project=image,
                                       save_conf=True,
                                       save_txt=True,
                                       save=True,
                                       device=0,
                                       max_det=5)
            # Move the saved predictions to the patient's destination folder
            path = os.path.join(const,image)
            print(path)
            shutil.move(os.path.join(const,image), patient_dest)
            
            del detections, temp_path, path
    

In [ ]:
perform_detection(root,destination)